# Lazy Theta Star Online

### Contents

1. **Introduction**
    1. [Why the problem is important](#intro)
    1. [What is lacking in the current knowledge / State of the art](#stateOfTheArt)
    1. [Objectives of the study & research question ](#researchQuestion)
    1. [Contribution](#contribution)

1. **Methods**
    1. [Installation](#installation)
    1. [Run](#run)
    1. [Study design](#studyDesign)
    1. [Study variables](#studyVariables)
    
1. [**Experimental Results**](#ExperimentalResults)
    1. [Open questions](#OpenQuestions)
    1. [Deterministic](#Deterministic)

<a id='intro'></a>
#### 1.A Why the problem is important

<a id='stateOfTheArt'></a>
#### 1.B What is lacking in the current knowledge / State of the art

Not comparing with MoveIt! because it is built for robotic arms and it would be a study in it's own right to find out the proper parameters. A nice playlist of intro videos exists. https://www.youtube.com/playlist?list=PLz8Xi5I9bhzKuFfcQxWvMm9sVCR5Wng90 

Another option is to use the underlying OMPL. But the obstacle avoidance is in MoveIt! .

Metrics from Helen and Lactombe 2002.

<a id='researchQuestion'></a>
#### 1.C Objectives of the study & research question

**Research question** What modifications need to be done to use Lazy theta * online and in the real world.



<a id='contribution'></a>
#### 1.D Contribution

- Online
- Receding horizon?
- Real world adaptation:
    - Free space corridor
    - Start & goal in same voxel
    - Unknown space as obstacle?
    - Scale?
    - Security margin & neighbor generation
- In case of Information Gain
    - Show with the sensor specs that the error os measurements is uniform throuhout the langer range.
    - [Disc area](https://www.mathopenref.com/arcsector.html)
    - [Is point part of arc](https://stackoverflow.com/questions/6270785/how-to-determine-whether-a-point-x-y-is-contained-within-an-arc-section-of-a-c)
    

**Security margin & neighbor generation** 

Using security margin to generate neighbors instead of octree resolution. [for calculations look in spreadsheet](https://docs.google.com/spreadsheets/d/1c4LzxBBLpTx6qQF26tpFr8tkmEM5zIi43MLb72PWlvE/edit#gid=0) 

When there is a large mismatch between security margin and the resolution there is no point generating neighbors for maximum resolution steps when most of them will be discarded. 

Instead a variable fraction of the security margin (this can be explored - know what is the amount of paths that cannot be found with this that can be found with resolution) (having as minimum lenght the octree resolution) will be sampled for neighbors. 

This reduces the number of neighbors while scaling with the size of the voxel. 
It is not possible to use this method of neighbor generation for both UpdateVertex and in the main loop because during the implementation of the algorithm the assumption that every maximum.

(Sparse version) Using the actual size of the neighbors did not in fact reduce computation time because if bookkeeping. 
    

**Obstacle avoidance - point generation**

Try the method using the cross product to generate an orthogonal vector. Can be pre generated at start and then applied to each point.

Add more distance to make sure of the margins also forward.

1. [The ortogonal coordinate frame can be found.](https://answers.ros.org/question/301060/how-to-use-tf-for-custom-rotation/)     

2. And then compute the transformation from one to the other:
    - [StackOverflow question](https://stackoverflow.com/questions/15252919/how-to-find-the-transformation-matrix-of-a-change-of-basis-with-eigen/15277421)    
    - [Pcl implementation of question](https://github.com/PointCloudLibrary/pcl/blob/master/common/include/pcl/common/impl/eigen.hpp#L927)     
    - [Pcl docs of function](http://docs.pointclouds.org/1.7.2/a01420.html#a71ec11d4f29c5c8a5b6220c0838a294f)
    - [Pcl unit tests](https://github.com/PointCloudLibrary/pcl/blob/cc7fe363c6463a0abc617b1e17e94ab4bd4169ef/test/common/test_eigen.cpp#L1018)
    - ? How to know which is the corresponding x axis.

**Filter laser**

The laser points aren't integrated fast engough to use a resolution of 0.2 in the ocotmap.     
Look into the filetring process to make it faster.

**Octomap diverges when receiving much information for an area**

Could be because of outliars.
Look into filtering. A good option could be median with a window around 3 to 5 points.


<a id='installation'></a>
#### 2.A Installation

There are 3 environments with px4: sitl, hitl and real world.
The instructions for installation are :
- stil  [Gazebo simulation - Try 9](Paper%20-%20Exploration%20real%20world.ipynb#try9)
- hitl [Hardware in the loop - Try 2 just upboard and laptop (no docker)](Paper%20-%20Exploration%20real%20world.ipynb#hitl)
    1. In laptop Firmware
        1. Install mavros-extras
        1. `cd ~`
        1. `wget https://raw.githubusercontent.com/PX4/Devguide/master/build_scripts/ubuntu_sim_ros_gazebo.sh`
        1. `chmod +x ubuntu_sim_ros_gazebo.sh`
        2. `source ubuntu_sim_ros_gazebo.sh`
        1. `git checkout v1.7.0-rc4`
        1. `make posix_sitl_default gazebo`
        1. `source ~/Flying_Octomap_code/devel/setup.bash`
        1. `source Tools/setup_gazebo.bash ~/src/Firmware ~/src/Firmware/build/posix_sitl_default`
        1. `export ROS_PACKAGE_PATH=$ROS_PACKAGE_PATH:~/src/Firmware`
        1. `export ROS_PACKAGE_PATH=$ROS_PACKAGE_PATH:~/src/Firmware/Tools/sitl_gazebo`
        1. `cp ~/Flying_Octomap_code/src/architecture/Tools/iris.xacro ~/src/Firmware/Tools/sitl_gazebo/models/rotors_description/urdf/`
        1. `cp ~/Flying_Octomap_code/src/architecture/Tools/iris_base.xacro ~/src/Firmware/Tools/sitl_gazebo/models/rotors_description/urdf/`
        1. `python ~/src/Firmware/Tools/sitl_gazebo/scripts/xacro.py -o ~/src/Firmware/Tools/sitl_gazebo/models/rotors_description/urdf/iris_base.urdf ~/src/Firmware/Tools/sitl_gazebo/models/rotors_description/urdf/iris_base.xacro enable_mavlink_interface:=true enable_ground_truth:=false enable_wind:=false enable_logging:=false rotors_description_dir:=/home/mfaria/src/Firmware/Tools/sitl_gazebo/models/rotors_description`
        1. `sed -i 's/package:\/\/rotors_description/package:\/\/mavlink_sitl_gazebo\/models\/rotors_description/g' ~/src/Firmware/Tools/sitl_gazebo/models/rotors_description/urdf/iris_base.urdf`
        1. `export ROS_IP=192.168.6.77`
        1. `export ROS_MASTER_URI=http://192.168.6.231:11311`
    1. In Upboard add `192.168.6.77 garuda` to `/etc/hosts`


- real world [Hardware Setup - Try 2 CATEC](Paper%20-%20Exploration%20real%20world.ipynb#try2_catec)

<a id='run'></a>
#### 2.B Run

###### Hardware in the Loop (HitL)
          
1. **Laptop** Setup basics for px4 simulation
    1. `cd ~/src/Firmware`
    1. `source /opt/ros/kinetic/setup.bash`
    1. `source ~/Flying_Octomap_code/devel/setup.bash`
    1. `source Tools/setup_gazebo.bash ~/src/Firmware ~/src/Firmware/build/posix_sitl_default`
    1. `export ROS_PACKAGE_PATH=$ROS_PACKAGE_PATH:~/src/Firmware`
    1. `export ROS_PACKAGE_PATH=$ROS_PACKAGE_PATH:~/src/Firmware/Tools/sitl_gazebo`
    1. `cp ~/Flying_Octomap_code/src/architecture/Tools/iris.xacro ~/src/Firmware/Tools/sitl_gazebo/models/rotors_description/urdf/`
    1. `cp ~/Flying_Octomap_code/src/architecture/Tools/iris_base.xacro ~/src/Firmware/Tools/sitl_gazebo/models/rotors_description/urdf/`
    1. `python ~/src/Firmware/Tools/sitl_gazebo/scripts/xacro.py -o ~/src/Firmware/Tools/sitl_gazebo/models/rotors_description/urdf/iris_base.urdf ~/src/Firmware/Tools/sitl_gazebo/models/rotors_description/urdf/iris_base.xacro enable_mavlink_interface:=true enable_ground_truth:=false enable_wind:=false enable_logging:=false rotors_description_dir:=/home/garuda/src/Firmware/Tools/sitl_gazebo/models/rotors_description`
    1. `sed -i 's/package:\/\/rotors_description/package:\/\/mavlink_sitl_gazebo\/models\/rotors_description/g' ~/src/Firmware/Tools/sitl_gazebo/models/rotors_description/urdf/iris_base.urdf`
    1. `export ROS_IP=192.168.6.77`
    1. `export ROS_MASTER_URI=http://192.168.6.231:11311`

1. **UpBoard** Launch code
    1. `source ~/ros_ws/devel/setup.bash`
    1. `export ROS_IP=192.168.6.231`
    1. `roslaunch architecture hitl_upboard_architecture.launch`

1. **Laptop** Launch px4 simulation (2 terminals)
    1. `cd ~/src/Firmware`
    1. `roslaunch architecture hitl_laptop_px4_simulation.launch`
    1. `roslaunch mavros px4.launch fcu_url:="udp://:14540@127.0.0.1:14557"`  

1. **Laptop** Launch rviz
    1. `cd ~/src/Firmware`
    1. `source /opt/ros/kinetic/setup.bash`
    1. `source ~/Flying_Octomap_code/devel/setup.bash`
    1. `source Tools/setup_gazebo.bash ~/src/Firmware ~/src/Firmware/build/posix_sitl_default`
    1. `export ROS_PACKAGE_PATH=$ROS_PACKAGE_PATH:~/src/Firmware`
    1. `export ROS_PACKAGE_PATH=$ROS_PACKAGE_PATH:~/src/Firmware/Tools/sitl_gazebo` 
    1. `export ROS_IP=192.168.6.77`
    1. `export ROS_MASTER_URI=http://192.168.6.231:11311`
    1. `rviz`
        
    


**Real Flight**

1. **Laptop** Setup basics for UAV model in gazebo & ips
    1. `cd ~/src/Firmware`
    1. `source /opt/ros/kinetic/setup.bash`
    1. `source ~/Flying_Octomap_code/devel/setup.bash`
    1. `source Tools/setup_gazebo.bash ~/src/Firmware ~/src/Firmware/build/posix_sitl_default`
    1. `export ROS_PACKAGE_PATH=$ROS_PACKAGE_PATH:~/src/Firmware`
    1. `export ROS_PACKAGE_PATH=$ROS_PACKAGE_PATH:~/src/Firmware/Tools/sitl_gazebo`
    1. `export ROS_IP=192.168.6.77`
    1. `export ROS_MASTER_URI=http://192.168.6.231:11311`

1. **UpBoard** Launch code
    1. `source ~/ros_ws/devel/setup.bash`
    1. `export ROS_IP=192.168.6.231`
    1. `roslaunch architecture realUav_upboard_px4comms.launch`
    1. `roslaunch architecture realUav_upboard_path_command.launch`

1. **Laptop** Setup basics for UAV model in gazebo & ips, Launch rviz
    1. `cd ~/src/Firmware`
    1. `source /opt/ros/kinetic/setup.bash`
    1. `source ~/Flying_Octomap_code/devel/setup.bash`
    1. `source Tools/setup_gazebo.bash ~/src/Firmware ~/src/Firmware/build/posix_sitl_default`
    1. `export ROS_PACKAGE_PATH=$ROS_PACKAGE_PATH:~/src/Firmware`
    1. `export ROS_PACKAGE_PATH=$ROS_PACKAGE_PATH:~/src/Firmware/Tools/sitl_gazebo` 
    1. `export ROS_IP=192.168.6.77`
    1. `export ROS_MASTER_URI=http://192.168.6.231:11311`
    1. `roslaunch architecture realUav_laptop_monitor.launch`
    1. get points: `rostopic echo /mavros/global_position/local` `rosrun tf tf_echo /map /base_link`
    1. send path request: `rostopic pub /ltstar_request path_planning_msgs/LTStarRequest`
    
Check the date and time are correct
- `date +%Y%m%d -s "20120418"`
- `date +%T -s "11:14:00"`

**Debug path on saved octree**
1. `roslaunch architecture debug_lazyThetaStar.launch`
1. `rosrun octomap_server octomap_server_node src/path_planning/test/data/d.bt _publish_free_space:=true`

**Valid bag data**
1. Experimenting 
    1. run realUav_laptop_monitor.launch
1. Generate octree
    1. Filter to exclude frontier and Lazy theta Star messages `rosbag filter 20180731_12_record_all_manualExploration_PathNotFound.bag 20180731_12_record_all_manualExploration_PathNotFound_scan.bag "topic == '/tf' or topic == '/scan' or topic == '/mavros/global_position/local' or topic == '/mavros/global_position/global' or topic == '/tf' or topic == '/mavros/global_position/local' or topic == '/mavros/global_position/global' or topic == '/mavros/setpoint_position/local' or topic == '/position_log'  or topic == '/scan'"`
    1. Make sure that async node is in STANDALONE mode and not savig csv files (defines)
    1. laptop@Terminal 1 `roslaunch architecture hitl_laptop_saveOctomap.launch`
    1. laptop@Terminal 2 `rosbag play 20180731_12_record_all_manualExploration_PathNotFound_scan.bag  -r 20`
    1. laptop@Terminal 3 `rosrun octomap_server octomap_saver karting.bt`
    1. Verify octree file `octovis karting.bt`
1. Use octree with rviz


**Extract data**

Deliverables:
- Octree
- Image with path
- Measurements in .csv file

Method:
1. run bag at rate 20 to identify point in time of request and take picture of the path.
2. filter the bag with tf and scan only
3. run `roslaunch architecture hitl_laptop_saveOctomap.launch` with reply of filtered bag, saving octree at identified points `rosrun octomap_server octomap_saver  file_name.bt`
4. run __ to verify output with filtered octree. Find valid path.
5. Add octree + start&goal as a test.

**Interpret RVIZ**

- Topic /ltstar_path. Maker array.
    - Input
        - start: small sphere (0.2m)         The exact start point lazyThetaStar_ method received.
        - goal: small sphere (0.2m)          The exact goal  point lazyThetaStar_ method received.
        - start_voxel: cube (size of voxel)  Voxel containing the start point.
        - goal_voxel : cube (size of voxel)  Voxel containing the goal  point.
    - Algorithm variables:
        - s     : ?                          The center of the voxels analyzed (s pseudocode variable)
        - closed: small green spheres (0.1m) The center of the voxels analyzed (s pseudocode variable)
    - Output
        - lazy_theta_star_path: series of yellow arrows that show the desired path. i.e.: straight lines between waypoints/voxel centers.
        - lazy_theta_star_wayoint: purple transparent cubes (size of voxel) Voxel containing each waypoint.
        - straight_line: (??lilac if found path, white otherwise) arrow from start to goal.
        

<a id='studyDesign'></a>
#### 2.C Study Design

Comparisions:
- It would be nice to compare against someone else's algorithm. MoveIt! seems to be the only 3D option of the shelf. See if this can be run from the bags. Also mention in future work that it will be compared - as it will...
- Regular grid & sparse?

20 runs from bag the see more statistically results.

Run around 3 in real life. Try to be with same flight.

Parameter sweep.

<a id='studyVariables'></a>
#### 2.D Study Variables

- Computation time
- RAM
- Path length free space
- Path lenght w/ obstacles: compare with manual measurement.
- ~~Failure percentage~~
    - ~~Naive max resolution neighbors~~
    - ~~Different fractions of security margin~~


**Run plotly**
1. Installation
    1. `pip install plotly`
    1. `pip install cufflinks`
1. Run frontier extraction for box plots
    1. `cd ~/Flying_Octomap_code/_generate_plots`
    1. `python`
    1. `variables= {}`
    1. `execfile( "compare_box_graphs.py", variables )`

<a id='ExperimentalResults'></a>
## 3 Experimental Results

**2018/08/21**      
Collected many runs.
Realized that the map deteriorates over time. Meaning the voxel separate instead of merging.


**2018/08/23**     
Collect video and some more runs.
As it is the UAV is not capable of integrating all the point clouds into the octree. Unclear if augmenting the resolution would help with voxels diverging.

It is proven that the current implementations works if the voxels are merged.

<a id='OpenQuestions'></a>
###### Filter Questions

1. If all the laser scans are properly integrated in an octomap with finer resolution will it still diverge?   
    **Yes**. When using a finer resolution it is apparent that one set of voxels that made the strutcture disapears to shitf to one side. This might be a precision problem because the RTK accuracy is only 0.9m or not if the accuracy of the RTK is only in respect to earth positioning and the diferencial between base and rover is always kept.

2. Can the filters 
    1. Be simplified (**yes**) to improve performance (yes) 
    2. Incorporate a median filter to remove outliars. In order to reduce octomap divergence. - Median slows down. Replaced by a intensity filter combined with tweaked parameters of sensor_model/[hit|miss] for the octomap.


In [17]:
# Intensities sample
import numpy as np


#intensities = [517.0, 520.0, 522.0, 523.0, 507.0, 534.0, 530.0, 525.0, 537.0, 526.0, 528.0, 526.0, 531.0, 519.0, 531.0, 524.0, 506.0, 533.0, 535.0, 543.0, 525.0, 536.0, 526.0, 559.0, 542.0, 535.0, 515.0, 526.0, 551.0, 556.0, 574.0, 564.0, 551.0, 540.0, 537.0, 556.0, 558.0, 549.0, 549.0, 551.0, 546.0, 551.0, 566.0, 528.0, 528.0, 521.0, 534.0, 518.0, 516.0, 527.0, 517.0, 534.0, 536.0, 505.0, 511.0, 535.0, 542.0, 544.0, 527.0, 521.0, 528.0, 532.0, 554.0, 556.0, 578.0, 591.0, 645.0, 448.0, 251.0, 357.0, 404.0, 391.0, 380.0, 414.0, 407.0, 413.0, 417.0, 437.0, 441.0, 410.0, 405.0, 433.0, 445.0, 472.0, 449.0, 449.0, 436.0, 429.0, 418.0, 447.0, 416.0, 430.0, 441.0, 421.0, 362.0, 369.0, 357.0, 374.0, 366.0, 378.0, 351.0, 326.0, 326.0, 322.0, 346.0, 345.0, 363.0, 378.0, 383.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 612.0, 595.0, 706.0, 706.0, 570.0, 131.0, 0.0, 413.0, 494.0, 642.0, 711.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 811.0, 811.0, 1119.0, 1229.0, 1416.0, 1416.0, 996.0, 944.0, 974.0, 997.0, 970.0, 834.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 471.0, 471.0, 501.0, 492.0, 496.0, 503.0, 503.0, 474.0, 493.0, 489.0, 520.0, 490.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 422.0, 421.0, 422.0, 430.0, 407.0, 408.0, 403.0, 403.0, 422.0, 470.0, 510.0, 532.0, 555.0, 580.0, 669.0, 675.0, 656.0, 664.0, 684.0, 736.0, 780.0, 752.0, 764.0, 752.0, 785.0, 777.0, 788.0, 761.0, 670.0, 593.0, 593.0, 0.0, 483.0, 508.0, 597.0, 617.0, 580.0, 571.0, 548.0, 536.0, 0.0, 0.0, 634.0, 634.0, 638.0, 588.0, 467.0, 415.0, 414.0, 409.0, 409.0, 412.0, 414.0, 415.0, 419.0, 411.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 387.0, 387.0, 452.0, 448.0, 426.0, 405.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 240.0, 334.0, 334.0, 0.0, 0.0, 299.0, 299.0, 0.0, 0.0, 369.0, 369.0, 403.0, 403.0, 418.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 240.0, 248.0, 228.0, 253.0, 264.0, 293.0, 257.0, 318.0, 318.0, 313.0, 347.0, 308.0, 357.0, 337.0, 350.0, 342.0, 407.0, 429.0, 433.0, 443.0, 434.0, 418.0, 340.0, 344.0, 323.0, 341.0, 336.0, 334.0, 376.0, 379.0, 389.0, 406.0, 398.0, 433.0, 454.0, 456.0, 436.0, 442.0, 442.0, 417.0, 418.0, 431.0, 431.0, 432.0, 438.0, 458.0, 440.0, 426.0]
#np.histogram(intensities, bins=[0, 1, 300, 400, 500, 600, 700, 800, 900, 1000, 1100, 1200, 1300])
#len(intensities)

ranges = [0.4309999942779541, 0.41100001335144043, 0.4230000078678131, 0.4129999876022339, 0.4320000112056732, 0.4169999957084656, 0.42100000381469727, 0.42399999499320984, 0.4269999861717224, 0.42800000309944153, 0.4350000023841858, 0.44200000166893005, 0.44200000166893005, 0.43700000643730164, 0.453000009059906, 0.4449999928474426, 0.4449999928474426, 0.4580000042915344, 0.47699999809265137, 0.47699999809265137, 0.4490000009536743, 0.46000000834465027, 0.48899999260902405, 0.47200000286102295, 0.4749999940395355, 0.4729999899864197, 0.46799999475479126, 0.46799999475479126, 0.4779999852180481, 0.49000000953674316, 0.49000000953674316, 0.4869999885559082, 0.4869999885559082, 0.5, 0.5180000066757202, 0.5180000066757202, 0.5189999938011169, 0.5049999952316284, 0.5149999856948853, 0.5109999775886536, 0.5109999775886536, 0.5120000243186951, 0.5170000195503235, 0.5199999809265137, 0.5360000133514404, 0.5379999876022339, 0.5379999876022339, 0.5389999747276306, 0.5519999861717224, 0.5529999732971191, 0.5460000038146973, 0.6100000143051147, 0.6480000019073486, 0.6430000066757202, 0.656000018119812, 0.656000018119812, 0.6539999842643738, 0.6520000100135803, 0.6650000214576721, 0.671999990940094, 0.671999990940094, 0.6700000166893005, 0.6700000166893005, 0.6959999799728394, 0.7020000219345093, 0.6990000009536743, 0.7039999961853027, 0.7110000252723694, 0.7260000109672546, 0.7310000061988831, 0.7390000224113464, 0.7429999709129333, 0.7480000257492065, 0.7609999775886536, 0.7590000033378601, 0.7739999890327454, 0.7689999938011169, 0.7879999876022339, 0.7919999957084656, 0.7950000166893005, 0.7990000247955322, 0.7960000038146973, 0.8149999976158142, 0.8240000009536743, 0.8339999914169312, 0.8510000109672546, 0.8460000157356262, 0.8560000061988831, 0.8700000047683716, 0.8830000162124634, 0.8960000276565552, 0.9100000262260437, 0.9160000085830688, 0.9330000281333923, 0.9380000233650208, 0.9570000171661377, 0.9570000171661377, 0.968999981880188, 0.9729999899864197, 1.0080000162124634, 1.0219999551773071, 1.0240000486373901, 1.0240000486373901, 1.0429999828338623, 1.0609999895095825, 1.0789999961853027, 1.0850000381469727, 1.1100000143051147, 1.1150000095367432, 1.1410000324249268, 1.156999945640564, 1.1740000247955322, 1.184000015258789, 1.2000000476837158, 1.215000033378601, 1.24399995803833, 1.25600004196167, 1.2769999504089355, 1.3079999685287476, 1.319000005722046, 1.3320000171661377, 1.371000051498413, 1.3849999904632568, 1.402999997138977, 1.434999942779541, 1.4570000171661377, 1.4880000352859497, 1.527999997138977, 1.5509999990463257, 1.562000036239624, 1.6180000305175781, 1.656000018119812, 1.722000002861023, 1.7940000295639038, 1.9040000438690186, 2.003999948501587, 2.111999988555908, 2.2260000705718994, 2.23799991607666, 2.2839999198913574, 2.2980000972747803, 2.315000057220459, 2.321000099182129, 2.365000009536743, 2.424999952316284, 2.4690001010894775, 2.4690001010894775, 2.4619998931884766, 2.434000015258789, 2.3440001010894775, 2.312999963760376, 2.2760000228881836, 2.249000072479248, 2.2679998874664307, 2.2809998989105225, 2.2869999408721924, 2.2869999408721924, 2.313999891281128, 2.4660000801086426, 3.2850000858306885, 3.380000114440918, 3.4149999618530273, 3.433000087738037, 3.433000087738037, 3.434000015258789, 3.430999994277954, 3.4230000972747803, 3.4230000972747803, 3.4230000972747803, 3.4260001182556152, 3.424999952316284, 3.4260001182556152, 3.4240000247955322, 3.4240000247955322, 3.4210000038146973, 3.4189999103546143, 3.4140000343322754, 3.4140000343322754, 3.4070000648498535, 3.4059998989105225, 3.4040000438690186, 3.4000000953674316, 3.4049999713897705, 3.4040000438690186, 3.4089999198913574, 3.4140000343322754, 3.4030001163482666, 3.3980000019073486, 3.4019999504089355, 3.4040000438690186, 3.4040000438690186, 3.4079999923706055, 3.4149999618530273, 3.4159998893737793, 3.4140000343322754, 3.4100000858306885, 3.3980000019073486, 3.3980000019073486, 3.4040000438690186, 3.4110000133514404, 3.4079999923706055, 3.3949999809265137, 3.3910000324249268, 3.382999897003174, 3.381999969482422, 3.384999990463257, 3.3980000019073486, 3.382999897003174, 3.421999931335449, 3.5179998874664307, 3.5399999618530273, 3.6029999256134033, 3.625999927520752, 3.8299999237060547, 3.8389999866485596, 3.8489999771118164, 3.8299999237060547, 3.8559999465942383, 3.9809999465942383, 4.072999954223633, 4.085999965667725, 4.0960001945495605, 4.0960001945495605, 4.263999938964844, 4.263999938964844, 4.089000225067139, 4.0269999504089355, 4.0279998779296875, 4.026000022888184, 3.9860000610351562, 3.9760000705718994, 3.7929999828338623, 3.7009999752044678, 3.5980000495910645, 3.5859999656677246, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 3.6559998989105225, 3.6559998989105225, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 3.74399995803833, 3.736999988555908, 3.684999942779541, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 3.9560000896453857, 3.9560000896453857, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 4.321000099182129, 4.317999839782715, 4.313000202178955, 4.326000213623047, 4.340000152587891, 4.348999977111816, 4.388999938964844, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 0.5270000100135803, 0.47699999809265137, 0.5180000066757202, 0.5149999856948853, 0.5049999952316284, 0.4779999852180481, 65.53299713134766, 65.53299713134766, 0.48100000619888306, 0.48100000619888306, 65.53299713134766, 65.53299713134766, 65.53299713134766, 0.45100000500679016, 0.45899999141693115, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 0.5289999842643738, 0.45899999141693115, 0.47699999809265137, 0.4490000009536743, 0.3779999911785126, 0.4339999854564667, 0.4359999895095825, 0.421999990940094, 0.38100001215934753, 0.38499999046325684, 0.375, 0.375, 65.53299713134766, 0.3089999854564667, 0.3370000123977661, 0.3050000071525574, 0.34200000762939453, 0.3070000112056732, 0.3019999861717224, 0.3540000021457672, 0.4050000011920929, 0.3799999952316284, 0.4390000104904175, 0.4169999957084656, 0.35899999737739563, 0.4320000112056732, 0.421999990940094, 0.4269999861717224, 0.42800000309944153, 0.4480000138282776, 0.4090000092983246, 0.46299999952316284, 0.7020000219345093, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 0.41499999165534973, 0.42899999022483826, 0.4320000112056732, 0.40700000524520874, 0.4099999964237213, 0.3790000081062317, 0.39899998903274536, 0.39899998903274536, 65.53299713134766, 0.34200000762939453, 0.34599998593330383, 0.3529999852180481, 0.335999995470047, 0.34700000286102295, 0.3370000123977661, 0.34700000286102295, 0.3490000069141388, 0.3490000069141388, 65.53299713134766, 65.53299713134766, 0.40700000524520874, 0.4090000092983246, 0.4359999895095825, 0.4650000035762787, 0.4659999907016754, 0.46399998664855957, 0.4620000123977661, 65.53299713134766, 0.4560000002384186, 0.4690000116825104, 0.4690000116825104, 0.46299999952316284, 0.46299999952316284, 0.4950000047683716, 0.4790000021457672, 0.503000020980835, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 0.5149999856948853, 0.5249999761581421, 0.5249999761581421, 65.53299713134766, 65.53299713134766, 65.53299713134766, 0.5509999990463257, 0.5509999990463257, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 65.53299713134766, 8.777999877929688, 8.777999877929688, 8.25100040435791, 8.25100040435791, 8.371999740600586, 8.319999694824219, 8.210000038146973, 8.177000045776367, 8.177000045776367, 8.116999626159668, 5.833000183105469, 3.555000066757202, 3.5209999084472656, 3.3910000324249268, 3.3910000324249268, 3.4790000915527344, 3.671999931335449, 3.681999921798706, 3.687000036239624, 3.8910000324249268, 3.885999917984009, 3.7090001106262207, 3.638000011444092, 3.5999999046325684, 3.565999984741211, 3.428999900817871, 3.325000047683716, 3.242000102996826, 3.1649999618530273, 3.072999954223633, 2.943000078201294, 2.875, 2.73799991607666, 2.5320000648498535, 2.5169999599456787, 2.440999984741211, 2.3519999980926514, 2.250999927520752, 2.187999963760376, 2.117000102996826, 2.0399999618530273, 1.9930000305175781, 1.944000005722046, 1.8819999694824219, 1.8209999799728394, 1.7890000343322754, 1.7389999628067017, 1.6920000314712524, 1.6490000486373901, 1.6139999628067017, 1.5779999494552612, 1.5540000200271606, 1.50600004196167, 1.4930000305175781, 1.4869999885559082, 1.4500000476837158, 1.406999945640564, 1.3869999647140503, 1.3769999742507935, 1.3680000305175781, 1.3489999771118164, 1.3240000009536743, 1.2940000295639038, 1.2790000438690186, 1.2489999532699585, 1.2300000190734863, 1.2120000123977661, 1.1959999799728394, 1.1859999895095825, 1.1720000505447388, 1.1549999713897705, 1.1440000534057617, 1.1299999952316284, 1.1139999628067017, 1.0789999961853027, 1.0540000200271606, 1.0540000200271606, 1.0549999475479126, 1.0529999732971191, 1.0290000438690186, 1.0089999437332153, 0.9990000128746033, 0.9869999885559082, 0.9739999771118164, 0.9639999866485596, 0.9580000042915344, 0.9350000023841858, 0.9200000166893005, 0.9150000214576721, 0.9039999842643738, 0.8920000195503235, 0.878000020980835, 0.878000020980835, 0.8930000066757202, 0.875, 0.8610000014305115, 0.8309999704360962, 0.8130000233650208, 0.8130000233650208, 0.8050000071525574, 0.7960000038146973, 0.8019999861717224, 0.7919999957084656, 0.7720000147819519, 0.7720000147819519, 0.7860000133514404, 0.7590000033378601, 0.7590000033378601, 0.7590000033378601, 0.7549999952316284, 0.7490000128746033, 0.7419999837875366, 0.7279999852180481, 0.7239999771118164, 0.722000002861023, 0.7059999704360962, 0.6990000009536743, 0.6930000185966492, 0.6890000104904175, 0.675000011920929, 0.6669999957084656, 0.6629999876022339, 0.6579999923706055, 0.6549999713897705, 0.6579999923706055, 0.656000018119812, 0.6179999709129333, 0.527999997138977, 0.5550000071525574, 0.5529999732971191, 0.5540000200271606, 0.5400000214576721, 0.5270000100135803, 0.5289999842643738, 0.5329999923706055, 0.5189999938011169, 0.5109999775886536, 0.5120000243186951, 0.4880000054836273, 0.4729999899864197, 0.4650000035762787, 0.4490000009536743, 0.44699999690055847, 0.453000009059906, 0.4440000057220459, 0.42500001192092896, 0.43299999833106995, 0.4449999928474426, 0.4309999942779541, 0.414000004529953, 0.42100000381469727, 0.3930000066757202, 0.39100000262260437, 0.36000001430511475, 0.32499998807907104, 0.3140000104904175, 0.3140000104904175, 0.2879999876022339, 0.27799999713897705, 0.2630000114440918, 0.26899999380111694, 0.28700000047683716, 0.30799999833106995, 0.3059999942779541, 0.3190000057220459, 0.335999995470047, 0.3400000035762787, 0.36800000071525574, 0.3630000054836273, 0.34700000286102295, 0.335999995470047, 0.33899998664855957, 0.33899998664855957, 0.35100001096725464, 0.36500000953674316, 0.37400001287460327, 0.39399999380111694, 0.40299999713897705, 0.39899998903274536, 0.39899998903274536, 0.39899998903274536, 0.38600000739097595, 0.38600000739097595, 0.39100000262260437, 0.414000004529953, 0.4189999997615814, 0.4169999957084656, 0.4050000011920929]
intensities = [464.0, 481.0, 491.0, 466.0, 468.0, 465.0, 499.0, 506.0, 515.0, 516.0, 509.0, 485.0, 485.0, 489.0, 505.0, 497.0, 497.0, 507.0, 508.0, 508.0, 495.0, 510.0, 533.0, 526.0, 536.0, 537.0, 543.0, 542.0, 534.0, 528.0, 527.0, 529.0, 528.0, 521.0, 515.0, 504.0, 506.0, 511.0, 546.0, 554.0, 554.0, 551.0, 532.0, 528.0, 525.0, 524.0, 524.0, 518.0, 519.0, 526.0, 518.0, 706.0, 934.0, 983.0, 1041.0, 1041.0, 1022.0, 1007.0, 1027.0, 1021.0, 1021.0, 1021.0, 1025.0, 1046.0, 1028.0, 1016.0, 1013.0, 1016.0, 1030.0, 1025.0, 1036.0, 1029.0, 1030.0, 1052.0, 1013.0, 1010.0, 1012.0, 1007.0, 1006.0, 1003.0, 1014.0, 1004.0, 1011.0, 1024.0, 1022.0, 1040.0, 1021.0, 1025.0, 1013.0, 1007.0, 1009.0, 1001.0, 1021.0, 993.0, 974.0, 1019.0, 1012.0, 1039.0, 1046.0, 1016.0, 1052.0, 1051.0, 1051.0, 1037.0, 1031.0, 1026.0, 974.0, 1027.0, 1003.0, 1006.0, 1003.0, 1006.0, 1025.0, 1015.0, 987.0, 981.0, 1007.0, 1011.0, 1005.0, 1009.0, 987.0, 990.0, 984.0, 974.0, 982.0, 980.0, 975.0, 957.0, 951.0, 942.0, 955.0, 937.0, 955.0, 978.0, 984.0, 975.0, 949.0, 937.0, 926.0, 945.0, 928.0, 937.0, 926.0, 921.0, 901.0, 920.0, 920.0, 938.0, 947.0, 939.0, 952.0, 976.0, 950.0, 985.0, 1067.0, 1213.0, 1213.0, 1291.0, 1344.0, 1005.0, 943.0, 937.0, 926.0, 926.0, 922.0, 927.0, 915.0, 916.0, 916.0, 914.0, 926.0, 938.0, 940.0, 940.0, 929.0, 932.0, 926.0, 926.0, 922.0, 935.0, 931.0, 919.0, 939.0, 932.0, 930.0, 934.0, 920.0, 911.0, 920.0, 930.0, 930.0, 941.0, 936.0, 942.0, 939.0, 949.0, 933.0, 933.0, 930.0, 949.0, 940.0, 934.0, 925.0, 919.0, 875.0, 881.0, 826.0, 692.0, 542.0, 440.0, 318.0, 340.0, 306.0, 453.0, 491.0, 620.0, 644.0, 666.0, 625.0, 533.0, 561.0, 604.0, 604.0, 382.0, 382.0, 500.0, 546.0, 553.0, 522.0, 482.0, 491.0, 367.0, 335.0, 257.0, 293.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 279.0, 279.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 285.0, 273.0, 281.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 321.0, 321.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 433.0, 461.0, 490.0, 514.0, 505.0, 468.0, 400.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 406.0, 414.0, 442.0, 438.0, 431.0, 432.0, 0.0, 0.0, 420.0, 420.0, 0.0, 0.0, 0.0, 407.0, 409.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 450.0, 424.0, 438.0, 436.0, 408.0, 471.0, 508.0, 538.0, 614.0, 618.0, 560.0, 560.0, 0.0, 538.0, 573.0, 538.0, 661.0, 701.0, 680.0, 614.0, 476.0, 410.0, 441.0, 586.0, 567.0, 711.0, 697.0, 686.0, 620.0, 545.0, 456.0, 400.0, 193.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 594.0, 630.0, 693.0, 674.0, 667.0, 578.0, 521.0, 521.0, 0.0, 522.0, 539.0, 601.0, 615.0, 628.0, 560.0, 542.0, 531.0, 531.0, 0.0, 0.0, 534.0, 521.0, 476.0, 436.0, 433.0, 427.0, 433.0, 0.0, 416.0, 415.0, 415.0, 404.0, 404.0, 421.0, 404.0, 421.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 403.0, 410.0, 410.0, 0.0, 0.0, 0.0, 261.0, 261.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 247.0, 247.0, 535.0, 535.0, 559.0, 549.0, 564.0, 549.0, 549.0, 493.0, 423.0, 273.0, 490.0, 583.0, 583.0, 552.0, 525.0, 512.0, 579.0, 1056.0, 1032.0, 1057.0, 1049.0, 992.0, 976.0, 962.0, 963.0, 958.0, 880.0, 834.0, 851.0, 849.0, 874.0, 902.0, 907.0, 893.0, 889.0, 895.0, 908.0, 932.0, 950.0, 956.0, 965.0, 954.0, 930.0, 919.0, 929.0, 958.0, 914.0, 912.0, 941.0, 946.0, 971.0, 966.0, 972.0, 969.0, 962.0, 957.0, 945.0, 968.0, 963.0, 986.0, 975.0, 1010.0, 988.0, 970.0, 984.0, 984.0, 981.0, 978.0, 997.0, 987.0, 978.0, 986.0, 993.0, 1010.0, 1010.0, 991.0, 985.0, 959.0, 971.0, 980.0, 1014.0, 1004.0, 987.0, 1001.0, 990.0, 986.0, 986.0, 989.0, 1007.0, 980.0, 980.0, 1024.0, 977.0, 994.0, 991.0, 1003.0, 1008.0, 1002.0, 995.0, 990.0, 1016.0, 1030.0, 1030.0, 1012.0, 1007.0, 1018.0, 1030.0, 1026.0, 1021.0, 1036.0, 1006.0, 1034.0, 1031.0, 1041.0, 1032.0, 1030.0, 1036.0, 1019.0, 1014.0, 1022.0, 1023.0, 1013.0, 1024.0, 905.0, 750.0, 535.0, 526.0, 514.0, 511.0, 509.0, 508.0, 512.0, 513.0, 506.0, 506.0, 516.0, 524.0, 567.0, 589.0, 563.0, 552.0, 548.0, 556.0, 562.0, 562.0, 565.0, 565.0, 570.0, 573.0, 579.0, 589.0, 574.0, 598.0, 611.0, 611.0, 610.0, 614.0, 623.0, 609.0, 608.0, 609.0, 606.0, 603.0, 578.0, 552.0, 554.0, 557.0, 564.0, 571.0, 580.0, 580.0, 544.0, 533.0, 522.0, 529.0, 532.0, 522.0, 535.0, 564.0, 558.0, 537.0, 525.0, 530.0, 545.0, 547.0, 533.0]
print len(intensities)


1081


In [15]:
## Calculation of sensor_model parameters

obstacle_size = 0.2
resolution = 0.5

delta_p_hit = 0.24 # € [0, 1] where 0 is very slow adding obstacles and 1 very fast
delta_p_miss = obstacle_size**3 * delta_p_hit / resolution**3

print "Delta probability hit: " + str(delta_p_hit)
print "Delta probability miss: " + str(delta_p_miss)

sensor_model_hit = 0.5 + delta_p_hit
sensor_model_miss = 0.5 - delta_p_miss

print "sensor_model/hit: " + str(sensor_model_hit)
print "sensor_model/miss: " + str(sensor_model_miss)

Delta probability hit: 0.24
Delta probability miss: 0.01536
sensor_model/hit: 0.74
sensor_model/miss: 0.48464


<a id='ResultSets'></a>
###### Result sets

Scenarios:
1. Karting (describe karting)
    1. Size
    1. Flat terrain except for some obstacles
    1. Obstacles: 
        1. large metallic structure (add picture) size x by x by x
        1. small tree
        1. many tires piled on the floor
        
1. Octrees
    1. 20180823_1110_raw collected at second 5 of bag 20180821_1110_manyRuns_karting_filterTfScan.bag . 
    This was after scanning an area of x by x by x there was much dust in the run. 
    1. 20180823_1110_filtered same bag and time point as previously but laser measurement intensity filter. Used only points with intensity € [0, 1] and [400, 2000].
    
- 20180821_1110_42712_raw.bt
- 20180821_1110_42936_raw.bt
- 20180821_1110_43042_raw.bt
- 20180821_1110_42712_filtered.bt --> too much dust
- 20180821_1110_42936_filtered.bt --> too much dust
- 20180821_1110_43042_filtered.bt --> too much dust
        
    
1. Testing conditions
    1. Live. The results were collected running the algorithm onboard the platform alongside data collection drivers, mapping generation and communication with px4.
    1. Hardware in the loop (hitl). The results were collected running the algorithm onboard the platform. Localization and mapping sourced from previous live flight.
    
1. Datasets   
Always safety margin of 2.5 to each side.
    1. 20180823_1110_raw ( -3.9; -14.76; 7) to (7.49; -6.98; 7) hitl
    1. 20180823_1110 filtered ( -3.9; -14.76; 7) to (7.49; -6.98; 7) hitl
    1. 20180823_1110_raw (7.49; -6.98; 7) to (-3.9; -14.76; 7) hitl
    2. 20180823_1110_filtered (7.49; -6.98; 7) to (-3.9; -14.76; 7) hitl

<a id='Deterministic'></a>
###### Deterministic

For fixed:
- octree
- start
- goal
- margin
The output is always the same.
The average time in hitl is x standard deviation y
The average time is live is w standard deviation z

In [12]:
exec(open("python/20180830_deterministic.py").read())

Analysis of time in millis by lenght of output path in meters. buckets by 10 m
 = 20.0 = 
Amount of data points 10
Standard deviation: 3634.78326727
Mean: 3957.0
Percentage: 35.684108259
Span: 10186 millis
 = 30.0 = 
Amount of data points 6
Standard deviation: 1794.37841296
Mean: 4688.33333333
Percentage: 44.837041803
Span: 4002 millis
 = 50.0 = 
Amount of data points 8
Standard deviation: 6863.0947454
Mean: 14320.625
Percentage: 36.6618309049
Span: 18720 millis


For each set of octree, start, goal and margin the output set of waypoints is always the same.
Each set was run x times.
The sets are those [above](#ResultSets)
The time the algorithm takes to reach the solution is 30348 millis +- 15326

<a id='Path_lenght_variability'></a>
###### Path lenght variability

- Switch start and end points.
- Keep start & end, change filter to non filter.
- Start & end points with similar obstacles.